In [203]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from unidecode import unidecode
from selenium.webdriver.common.keys import Keys
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime

driver = webdriver.Chrome('webdriver/chromedriver')
WAIT_TIME = 30

In [204]:
emails = pd.read_csv("company_domains.txt", sep=":", header=None, names="domain status".split())

In [205]:
ok_emails = emails.loc[emails.status=='ok']["domain"].tolist()

In [206]:
for company_domain in ok_emails[:10]:
    
    print("searching for {}...".format(company_domain))
    
    comp_info = {}
    
    driver.get("https://www.google.com.au/")
    SUBJ_WORD = " ".join(["linkedin", company_domain])
    WebDriverWait(driver, WAIT_TIME).until(EC.presence_of_element_located((By.XPATH, "//input[@id='lst-ib']"))).send_keys(SUBJ_WORD)
    driver.find_element_by_xpath("//input[@type='submit']").click()

    h3 = driver.find_element_by_xpath("//div[@class='s']")
    linkedin_url = h3.find_element_by_xpath(".//cite[@class='_Rm']").text
    
    if "au.linkedin.com" in linkedin_url:
        driver.get(linkedin_url)
        
        try:
            WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, "public-follow")))
        except:
            print("no follow button, skipping..")
            continue
            
        try:
            cname = driver.find_element_by_xpath("//h1[@class='name']")
        except:
            # we don't care about companies without names
            continue
        
#         li_part = linkedin_url.split("/")[-1] 
        
#         print("are these: ", li_part.split("-"), " in ", company_domain.lower().strip())
        
#         if sum([1 if w in company_domain.lower().strip() else 0 for w in li_part.split("-")]) == 0:
#             continue
        
        try:
            indus = driver.find_element_by_xpath("//p[@class='industry']")
        except:
            indus = None
            
        try:
            csize = driver.find_element_by_xpath("//p[@class='company-size']")
        except:
            size = None
            
        try:
            cwebs = driver.find_element_by_xpath("//li[@class='website']/p")
        except:
            cwebs = None
        
        try:
            ctype = driver.find_element_by_xpath("//li[@class='type']/p")
        except:
            ctype = None
        
        try:
            cadr = driver.find_element_by_xpath("//p[@class='adr']")
        except:
            cadr = None
            
        comp_info.update({k: v.text.lower().strip() if v else None for k, v in {"name": cname, "industry": indus, "size": csize,
                                "website": cwebs, "type": ctype, "address": cadr}.items()})
    
        print(comp_info)
        
    else:
        print("can\'t find this company on linkedin, moving on...".format(company_domain.upper()))     
    
driver.quit()

searching for churchgrace.com.au...
{'industry': 'law practice', 'name': 'church & grace', 'type': 'privately held', 'address': '2000 australia', 'size': '11-50 employees', 'website': 'http://www.churchgrace.com.au'}
searching for groundsupportservices.com...
{'industry': 'outsourcing/offshoring', 'name': 'flat planet', 'type': 'privately held', 'address': 'level 6, 115 pitt street sydney, nsw 2000 australia', 'size': '51-200 employees', 'website': 'http://www.flatplanet.com.au'}
searching for lams.com.au...
can't find this company on linkedin, moving on...
searching for webtrix.com.au...
{'industry': None, 'name': 'webtrix, llc', 'type': None, 'address': '313 caraway dr bear, delaware 19701 united states', 'size': '11-50 employees', 'website': 'http://webtrix.net'}
searching for royaland.com.au...
no follow button, skipping..
searching for csideplumbing.com.au...
no follow button, skipping..
searching for irvine.id.au...
no follow button, skipping..
searching for babygenerationcc.com.